# Classifying MNIST with MLflow

This is a demo for using the MLflow machine learning library to classify digits in the MNIST dataset. 

## Imports

In [10]:
# main library
import mlflow as mf
import numpy as np
# to run this example, you have to install python-mnist and scikit learn
# use "pip install python-mnist"
from mnist import MNIST
from sklearn.preprocessing import StandardScaler
# use "pip install sklearn"
from sklearn.utils import shuffle
# for progress bar
from tqdm import tqdm
from matplotlib import pyplot as plt

/Users/macbook/miniconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1120: UserWarning: Bad val "$TEMPLATE_BACKEND" on line #41
	"backend      : $TEMPLATE_BACKEND
"
	in file "/Users/macbook/.matplotlib/matplotlibrc"
	Key backend: Unrecognized backend string "$template_backend": valid strings are [u'pgf', u'ps', u'Qt4Agg', u'GTK', u'GTKAgg', u'nbAgg', u'agg', u'cairo', u'MacOSX', u'GTKCairo', u'Qt5Agg', u'template', u'WXAgg', u'TkAgg', u'GTK3Cairo', u'GTK3Agg', u'svg', u'WebAgg', u'pdf', u'gdk', u'WX']
  (val, error_details, msg))


## Generating Dataset and Setting Hyperparameters

In [2]:
# for testing consistency
np.random.seed(42)

In [12]:
### data set
mndata = MNIST('./dataset')
x_train, y_train = mndata.load_training()
x_test, y_test = mndata.load_testing()

# make a test image for visualization
test_img = x_train[0]

# make them numpy arrays
x_train, _y_train, x_test, _y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

In [4]:
# hyperparams
l_rate = 0.5
s_rate = 0.2
training_size = x_train.shape[0]
test_size = x_test.shape[0]
batch_size = 50
n_batches = training_size / batch_size
epochs = 10
image_size = x_train.shape[1]
n_labels = 10

In [5]:
# onehot encode labels
y_train = np.zeros((training_size, n_labels))
y_test = np.zeros((test_size, n_labels))
# standarize data
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.transform(x_test)

for y_idx in range(training_size):
	hot_idx = _y_train[y_idx]
	y_train[y_idx, hot_idx] = 1

for y_idx in range(test_size):
	hot_idx = _y_test[y_idx]
	y_test[y_idx, hot_idx] = 1

print "Data preparation compelete..."


Data preparation compelete...


/Users/macbook/miniconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


## Creating a Computation Graph

In [6]:
# get the graph from pickle
g = mf.Graph()

### input data place holder
x_placeholder = mf.placeholder([batch_size, image_size], graph = g)
y_placeholder = mf.placeholder([batch_size, n_labels], graph = g)

### NN layer 1
W1 = mf.Variable(np.random.random((image_size, 200)), graph = g)
b1 = mf.Variable(np.zeros((200)), graph = g) 

### activation
z1 = mf.matmul(x_placeholder, W1, graph = g) + b1
a1 = mf.sigmoid(z1, graph = g)

### NN layer 2
W2 = mf.Variable(np.random.random((200, n_labels)), graph = g)
b2 = mf.Variable(np.zeros((n_labels)), graph = g) 

### activation
z2 = mf.matmul(a1, W2, graph = g) + b2
pred = mf.sigmoid(z2, graph = g)


### cost function
cost = mf.sum(pow((pred - y_placeholder), 2.0), graph = g) / (2 * batch_size)

# optimizer
opt = mf.Train.AdaGradOptimizer(l_rate, graph = g).minimize(cost)

## Optimization

In [7]:
# create session and run prediction!
sess = mf.Session(g)
for e in range(epochs):
	print "Epoch " + str(e)
	print "Cost: ", sess.run(cost, feed_dict = {x_placeholder: x_train[:batch_size], y_placeholder: y_train[:batch_size]})
	for b in tqdm(range(n_batches)):
		sess.run(opt, feed_dict = {x_placeholder: x_train[b * batch_size:(b + 1) * batch_size], y_placeholder: y_train[b * batch_size:(b + 1) * batch_size]})
	x_train, y_train = shuffle(x_train, y_train, random_state=42)

print "50 Sample Train Cost: \n", sess.run(cost, feed_dict = {x_placeholder: x_train[:batch_size], y_placeholder: y_train[:batch_size]})
print "50 Sample Test Cost: \n", sess.run(cost, feed_dict = {x_placeholder: x_test[:batch_size], y_placeholder: y_test[:batch_size]})


  4%|▍         | 50/1200 [00:01<00:22, 50.55it/s]/Users/macbook/miniconda2/lib/python2.7/site-packages/mlflow/Ops/linalg.py:10: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-a))
100%|██████████| 1200/1200 [00:21<00:00, 56.24it/s]

Epoch 0
Cost:  2.81334710162
Epoch 1
Cost:  


100%|██████████| 1200/1200 [00:29<00:00, 34.48it/s]

0.181512015844
Epoch 2
Cost:  


100%|██████████| 1200/1200 [00:23<00:00, 51.88it/s]

...

0.153223010334
Epoch 8
Cost:  


100%|██████████| 1200/1200 [00:20<00:00, 59.06it/s]

0.128402926473
Epoch 9
Cost:  


100%|██████████| 1200/1200 [00:19<00:00, 60.53it/s]

0.0908914086771
50 Sample Train Cost: 
0.163068355402
50 Sample Test Cost: 
0.110391132192
